In [1]:
import numpy as np

import astropy
import astropy.units as ur
import astropy.constants as cr

%matplotlib inline

from astropy.io import fits
from specutils import Spectrum1D
from specutils.manipulation import (box_smooth, gaussian_smooth, trapezoid_smooth)

from pylab import *

import scipy.interpolate as II

In [98]:
fλ_unit = ur.erg/ur.s/ur.cm**2/ur.Angstrom # Spectral radiances per Hz or per angstrom
fν_unit = ur.erg/ur.s/ur.cm**2/ur.Hertz

Ω_arcsecond = 1/(3600.**2 * 57.4**2) * ur.sr

µ = ur.micron
arcsec = ur.arcsec
nm = ur.nm

In [99]:
def fnum_to_steradian(fnum):
    " See eq. 24 https://www.labsphere.com/site/assets/files/2551/integrating_sphere_theory_apps_tech_guide.pdf"
    
    return np.pi/(4*fnum*fnum) * ur.steradian


def steradian_to_fnum(sr):
    
    return np.sqrt(np.pi/(4*sr/ur.steradian))

## Inputs

In [142]:
pixel_size = 10 * µ
plate_scale = 6 * arcsec
RN = 1 
BGDs = np.array([22, 22, 22]) * ur.ABmag
target = 22 * ur.ABmag
exptime = 20*ur.second
efficiency = 0.5

bands = np.array([ 
    (180, 220),
    (240, 300),
    (340, 380)
        ])*nm

## Outputs

In [168]:


for bgd, band in zip(BGDs, bands):
    λ_mid = np.mean(band)
    epp = (cr.c*cr.h/λ_mid).cgs
    BW = np.abs(band[0]-band[1])
    
    A_pixel = pixel_size**2
    pixel_angular_area = (plate_scale**2 / ur.arcsecond**2).cgs
    
    # Spectral radiance in band --> # photons in band
    L_λ_bgd = bgd.to(flu, equivalencies=ur.spectral_density(λ_mid)) / Ω_arcsecond 
    L_λ_obj = target.to(flu,equivalencies=ur.spectral_density(λ_mid)) / Ω_arcsecond 
    
    num_photons_bgd = (L_λ_bgd * exptime * BW * efficiency * pixel_angular_area * A_pixel / epp).to(1/ur.steradian)
    
    bgd_limit = 10 * RN
    required_Ω = bgd_limit / num_photons_bgd
    
    fnum = steradian_to_fnum(required_Ω)
    focal_length = (pixel_size/(plate_scale/ur.arcsec) * 206265).to(ur.mm)
    Dtel = (focal_length/fnum).to(ur.cm)
    
    print()
    print("At {0} the energy per photon is {1:1.1e}".format(λ_mid, epp))
    print("Per pixel the number of photons per Sr are {0:5.1e}".format(num_photons_bgd))
    print("Resul f/{0:3.1f} with f={1:3.1f} ∴ Dtel={2:3.1f}".format(fnum, focal_length, Dtel))


At 200.0 nm the energy per photon is 9.9e-12 erg
Per pixel the number of photons per Sr are 2.7e+04 1 / sr
Required focal ratio is f/45.8 with f=343.8 mm ∴ Dtel=0.8 cm

At 270.0 nm the energy per photon is 7.4e-12 erg
Per pixel the number of photons per Sr are 3.0e+04 1 / sr
Required focal ratio is f/48.3 with f=343.8 mm ∴ Dtel=0.7 cm

At 360.0 nm the energy per photon is 5.5e-12 erg
Per pixel the number of photons per Sr are 1.5e+04 1 / sr
Required focal ratio is f/34.1 with f=343.8 mm ∴ Dtel=1.0 cm
